# Исследование надежности заемщиков.

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

# Шаг 1. Обзор данных

Получим начальное представление о имеющейся у нас БД:

In [1]:
import pandas as pd # импорт библиотеки pandas
data = pd.read_csv('/datasets/data.csv') # чтение файла с данными и сохранение в data

In [2]:
display(data.head(10)) # получение первых 10 строк таблицы df

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [3]:
data.info() # получение общей информации о данных в таблице data

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Во-первых**, наблюдаем явную проблему с колонкой `days_employed`.

* Дни не могут быть вещественным числом.
* Дни не могут быть отрицательными.

Тут похоже есть проблема с выгрузкой или самим заполнением таблицы. Следовало бы задать пару вопросов человеку, который эти данные формировал. Предварительно можно решить эту проблему, взяв числа по модулю и перевести их в целочисленные значения.


**Во-вторых**, заметны дубликаты в столбце `education`.

* Одни и те же слова записаны в разном регистре

Не беда, приведем их к нижнему регистру!

**В-третьих**, по общей информации видим большое количестов пропусков в стобцах `days_employed` и `total_income`.

# Шаг 2.1 Заполнение пропусков 

Имеем большое количество пропусков в двух стобцах. Для начала посчитаем их количество, и удостоверимся, что в других стобцах пропусков нет.

In [4]:
empty_values = data.isna().sum() # подсчёт пропусков
empty_values

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Можем подсчитать, какую долю составляют пропущенные значения в каждом из столбцов с пропусками.

In [5]:
column = 'days_employed'
percent_miss_values = data[column].isna().sum() / data[column].shape[0] # количество пропусков делим на общее количество строк в стобце
print(f'{percent_miss_values:.2%}')

10.10%


In [6]:
column = 'total_income'
percent_miss_values = data[column].isna().sum() / data[column].shape[0] # количество пропусков делим на общее количество строк в стобце
print(f'{percent_miss_values:.2%}')

10.10%


Видим, что в двух колонках процент пустых значений одинаков и составляет 10%. Не такое уж и маленькое значение, 
когда речь идет об анализе платёжеспособности клиентов. Так что лучше будет обработать эти пропуски и заполнить их медианными 
значениями, ведь они адекватно отражают среднюю тенденцию для ассимитричных распределений.

Однако, для того, чтобы вычислить медиану адекватно, сначала избавимся от отрицательных значений в стобце `days_employed` и
, на всякий случай, в `total_income`, а так же переведем дни в целочисленное значение.

In [7]:
data['days_employed'] = abs(data['days_employed']) # берем значения по модулю
data['total_income'] = abs(data['total_income'])

Теперь имеем "нормальные" значения

In [8]:
data['days_employed'] = data['days_employed'].fillna(data['days_employed'].median()) # заполняем пропуски медианами
data['total_income'] = data['total_income'].fillna(data['total_income'].median())

Готово! Осталось проверить, пропали ли пропуски?

In [9]:
empty_values = data.isna().sum() # подсчёт пропусков
empty_values

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

In [10]:
display(data.head(10)) # посмотрим, что все красиво

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


С этим разобрались! Что дальше?

 # Шаг 2.2 Проверка данных на аномалии и исправления.

Самое время разобраться с другими столбцами и поискать еще нелепости...
Будем использовать функцию `value_counts()`

**Проблемы, которые были выявлены:**
* `data['children']` → Отрицательное количесвто детей. Неправдоподобно большое количество детей (20). 
* `data['dob_years']` → 101 сторока с показателем возраста в "0" лет. К счастью, нам возраст и не важен!
* `data['education']` → Огромное количесвто одинаковых строк, вот только регистр разный.
* `data['family_status']` → "Не женат / не замужем" немного режет глаз заглавная "Н", но проблем сданными это не вызывает.
* `data['gender']` →  ̶X̶N̶A̶ ̶-̶ ̶Е̶щ̶е̶ ̶о̶д̶и̶н̶ ̶н̶о̶в̶ы̶й̶ ̶г̶е̶н̶д̶е̶р̶?̶  Для этого анализа гендер не имеет значения. XNA не повлияет на него.
* `data['purpose']` → "ремонт жилью" - очевидная опечатка.

И того, нам нужно обработать 3 столбца: 
* `data['children']` Скорее всего 20 детей - это 2, просто с лишним ноликом. А -1 возьмем за 1 ребенка.

In [11]:
# исправим значения 20 для возраста, согласно нашей гипотезе
data.loc[data['children'] == 20, 'children'] = 2
# исправим значение -1 для количества детей, согласно нашей гипотезы 
data.loc[data['children'] == -1, 'children'] = 1
# и сразу проверим
data['children'].value_counts()


0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

* `data['education']` Приведем все строки в нижний регистр.

In [12]:
data['education'] = data['education'].str.lower()
# заодно и надоедливую "Н" прировняем ко всем!
data['family_status'] = data['family_status'].str.lower()
# так же проверяем
data['education'].value_counts()

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

* `data['purpose']` Правильно не "ремонт жилью", а "ремонт жилья".

In [13]:
# исправим опечатку в purpose
data.loc[data['purpose'] == 'ремонт жилью', 'purpose'] = 'ремонт жилья'
# проверим, что все хорошо
data['purpose'].sort_values().value_counts()

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
операции с жильем                         653
покупка жилья для сдачи                   653
операции с коммерческой недвижимостью     651
жилье                                     647
покупка жилья                             647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
строительство недвижимости                620
покупка своего жилья                      620
ремонт жилья                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

# Шаг 2.3. Изменение типов данных.

Заменим тип `total_income` на int, т.к. точность до копеек нам сейчас не интересна, поскольку нам нужно конкретный целочисленный тип, а не вещественный, используем именно `astype`. Вместе с этим поправим и вещественное количество дней. 

In [14]:
data['total_income'] = data['total_income'].astype('int')
data['days_employed'] = data['days_employed'].astype('int')

# проверим, что все сделали верно
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21525 non-null  int64 
 1   days_employed     21525 non-null  int64 
 2   dob_years         21525 non-null  int64 
 3   education         21525 non-null  object
 4   education_id      21525 non-null  int64 
 5   family_status     21525 non-null  object
 6   family_status_id  21525 non-null  int64 
 7   gender            21525 non-null  object
 8   income_type       21525 non-null  object
 9   debt              21525 non-null  int64 
 10  total_income      21525 non-null  int64 
 11  purpose           21525 non-null  object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


# Шаг 2.4. Удаление дубликатов.

Проверим, есть ли дубликаты в таблице с помощью метода `duplicated` и избавимся от всего лишнего.

In [15]:
data.duplicated().sum()

71

In [16]:
# удаляем дубликаты
data.drop_duplicates(subset=None, keep="first", inplace=True)
# и проверим, что все хорошо
data.duplicated().sum()

0

# Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

Имеет сымысл создать два новыйх датафрейма для того, чтобы разгрузить нашу основную таблицу, и при этом все еще иметь доступ к тем данным, которые мы уберем. По сути создаем словари. Уберем два стобца `education` и `family_status` т.к. имеем их индификаторы.

In [17]:
# создаем новый датафрейм
education_df = data[['education', 'education_id']]
education_df.value_counts()

education            education_id
среднее              1               15172
высшее               0                5250
неоконченное высшее  2                 744
начальное            3                 282
ученая степень       4                   6
dtype: int64

In [18]:
# создаем новый датафрейм
family_df = data[['family_status', 'family_status_id']]
family_df.value_counts()

family_status          family_status_id
женат / замужем        0                   12339
гражданский брак       1                    4151
не женат / не замужем  4                    2810
в разводе              3                    1195
вдовец / вдова         2                     959
dtype: int64

In [19]:
# убираем уже не нужные столбцы
data = data.drop(columns='family_status')
data = data.drop(columns='education')
# проверка 
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21454 entries, 0 to 21524
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21454 non-null  int64 
 1   days_employed     21454 non-null  int64 
 2   dob_years         21454 non-null  int64 
 3   education_id      21454 non-null  int64 
 4   family_status_id  21454 non-null  int64 
 5   gender            21454 non-null  object
 6   income_type       21454 non-null  object
 7   debt              21454 non-null  int64 
 8   total_income      21454 non-null  int64 
 9   purpose           21454 non-null  object
dtypes: int64(7), object(3)
memory usage: 1.8+ MB


Визульно разгрузили нашу основную таблицу и не потеряли ни капли информации!

# Шаг 2.6. Категоризация дохода.

Для облегчтения работы с доходами клинтов банка следует создать новый столбец, где клиенты будут делиться на 5 групп по количеству зароботной платы. 

In [20]:
# Создаем функцию для сортировки прибыли на группы
def income_level(income):
    if income <=30000: 
        return 'E'
    elif income <= 50000: 
        return 'D'
    elif income <= 200000:
        return 'C'
    elif income <= 1000000:
        return 'B'
    elif income >= 1000001:
        return 'A'
 
# создаем список отсортированных групп для последущего превращения его в колонку
list_income_level = []
for income in data['total_income']:
    list_income_level.append(income_level(income))

data.insert(9, 'total_income_category', list_income_level) # используя метод 'insert', вставляем столбец в нужное нам место
display(data.head(10)) # проверка
display(data['total_income_category'].value_counts()) # посмотрим каких клиентов, с каким уровнем зарпалты больше всего

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,total_income_category,purpose
0,1,8437,42,0,0,F,сотрудник,0,253875,B,покупка жилья
1,1,4024,36,1,0,F,сотрудник,0,112080,C,приобретение автомобиля
2,0,5623,33,1,0,M,сотрудник,0,145885,C,покупка жилья
3,3,4124,32,1,0,M,сотрудник,0,267628,B,дополнительное образование
4,0,340266,53,1,1,F,пенсионер,0,158616,C,сыграть свадьбу
5,0,926,27,0,1,M,компаньон,0,255763,B,покупка жилья
6,0,2879,43,0,0,F,компаньон,0,240525,B,операции с жильем
7,0,152,50,1,0,M,сотрудник,0,135823,C,образование
8,2,6929,35,0,1,F,сотрудник,0,95856,C,на проведение свадьбы
9,0,2188,41,1,0,M,сотрудник,0,144425,C,покупка жилья для семьи


C    16016
B     5041
D      350
A       25
E       22
Name: total_income_category, dtype: int64

Выполнив все действия, можем заметить, что клинтов категории "С", то есть зарабатывающих от 50001 до 200000, больше всего.

# Шаг 2.7. Категоризация целей кредита.

Создадим бОльшие группы для работы со столбцом `purpose` и его категоризации.

In [21]:
data['purpose'] = data['purpose'].astype('string') # переведем значения сталбца в строки, для последущей обработки

In [22]:
# создаем фунцию, которая возвращает нужную нам строку в зависимости от наличя подстроки в строке столбца purpose
def category_purpose(row):
    purpose = row['purpose']
    if 'недвижим' in purpose:
        return 'операции с недвижимостью'
    elif 'жил' in purpose:
        return 'операции с недвижимостью'
    elif 'свадьб' in purpose:
        return 'проведение свадьбы'
    elif 'автомоб' in purpose:
        return 'операции с автомобилем'
    elif 'образова' in purpose:
        return 'получение образования'
    else:
        return 'неопределено'

data['purpose_category'] = data.apply(category_purpose, axis=1) # создаем столбец
data['purpose_category'].value_counts() # проверяем


операции с недвижимостью    10811
операции с автомобилем       4306
получение образования        4013
проведение свадьбы           2324
Name: purpose_category, dtype: int64

Наблюдаем, что операций с недвижимостью больше всего, в то время как кредиты на свадьбу берут меньше всего. Кредиты на образование и операции с автомобилем примерно одинаковое значение. 

# Шаг 3. Ответы на вопросы.

**Вопрос 1:** Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [23]:
debt_from_children = pd.DataFrame()
debt_from_children['count_children'] = data.groupby('children')['debt'].count()
debt_from_children['sum_children'] = data.groupby('children')['debt'].sum()
debt_from_children['result_children'] = debt_from_children['sum_children'] / debt_from_children['count_children'] 
debt_from_children.sort_values(by='result_children', ascending = False)

,count_children,sum_children,result_children
children,,,
4,41,4,0.097561
2,2128,202,0.094925
1,4855,445,0.091658
3,330,27,0.081818
0,14091,1063,0.075438
5,9,0,0.000000


**Вывод 1:** С увеличением количества детей мы видим увеличение количества просроченных задолженностей, хотя люди с 3 детьми чаще платят в срок чем люди с 1 ребенком. Бездетные, как правило реже просрачивают оплату по кредиту, чем люди с детьми.

**Вопрос 2:** Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [24]:
debt_from_family_status = pd.DataFrame()
debt_from_family_status['count_family_status'] = data.groupby('family_status_id')['debt'].count()
debt_from_family_status['sum_family_status'] = data.groupby('family_status_id')['debt'].sum()
debt_from_family_status['result_family_status'] = debt_from_family_status['sum_family_status'] / debt_from_family_status['count_family_status'] 
debt_from_family_status.sort_values('result_family_status', ascending = False)

,count_family_status,sum_family_status,result_family_status
family_status_id,,,
4,2810,274,0.097509
1,4151,388,0.093471
0,12339,931,0.075452
3,1195,85,0.071130
2,959,63,0.065693


**Вывод 2:** Да, зависимость есть. Люди не в браке и не бывавшие в браке имеют больший процент невозвратов в срок. Но, те кто развелись или овдовели чаще платят в срок, чем люди в браке.

**Вопрос 3:** Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [25]:
debt_from_total_income = pd.DataFrame()
debt_from_total_income['count'] = data.groupby('total_income_category')['debt'].count()
debt_from_total_income['sum'] = data.groupby('total_income_category')['debt'].sum()
debt_from_total_income['conversion'] = debt_from_total_income['sum'] / debt_from_total_income['count']
debt_from_total_income.sort_values('conversion', ascending = False)

,count,sum,conversion
total_income_category,,,
E,22,2,0.090909
C,16016,1360,0.084915
A,25,2,0.080000
B,5041,356,0.070621
D,350,21,0.060000


**Вывод 3:** На самом деле в данном случае будет несправедливо сравнить результаты, ибо основную часть людей представляют доход "С" категории, однако в целом видим, что те, кто зарабатывают от 30000 до 50000 имеют меньше проскрочек по платежам.

**Вопрос 4:** Как разные цели кредита влияют на его возврат в срок?

In [27]:
debt_from_purpose_category = pd.DataFrame()
debt_from_purpose_category['count_purpose_category'] = data.groupby('purpose_category')['debt'].count()
debt_from_purpose_category['sum_purpose_category'] = data.groupby('purpose_category')['debt'].sum()
debt_from_purpose_category['result_purpose_category'] = debt_from_purpose_category['sum_purpose_category'] / debt_from_purpose_category['count_purpose_category'] 
debt_from_purpose_category.sort_values('result_purpose_category', ascending = False)

,count_purpose_category,sum_purpose_category,result_purpose_category
purpose_category,,,
операции с автомобилем,4306,403,0.093590
получение образования,4013,370,0.092200
проведение свадьбы,2324,186,0.080034
операции с недвижимостью,10811,782,0.072334


**Вывод 4:** Как мы видим, чаще всего платят в срок с целью кредита недвижимость, просрочка у тех кому нужен кредит на авто и образование. Кредиты на свадьбу просрачивают реже - примерно как и в случае недвижимости.

# Общий вывод:

Проанализировав полученные данные, мы исправили некорректные строки в базе данных, отредактировали неточности и опечатки, приведя таблицу в читаемый вид. Проработав все эти нюансы, стало возможно проверить гипотезы. Результаты проверки следующие:
1. Семейный статус влияет на вероятность платежей по кредиту в срок - люди, бывавшие в браке чаще плятят в срок чем те, кто не бывали в браке, причем разведенные и овдовевшие платят в срок чаще, чем люди в браке. 
2. Чем больше детей, тем чаще платят не в срок, бездетные реже просрочивают оплату.